Installing libraries:

In [2]:
%load_ext autoreload
%autoreload 2

import os
import re
import sys
import numpy as np
import json
from pydub import AudioSegment, silence

from IPython.display import Audio, display

rootdir_path = '/home/christosf/AttProtos'
sys.path.append(rootdir_path)
from apnet.datasets import MUSDB18

/home/christosf/anaconda3/envs/apnet2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/christosf/anaconda3/envs/apnet2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/christosf/anaconda3/envs/apnet2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/christosf/anaconda3/envs/apnet2

In [33]:
# play audio file. If list, play all audio files
def play_audio(path_of_wav):
    if isinstance(path_of_wav, list):
        for item in path_of_wav:
            display(Audio(item))
    else:
        display(Audio(path_of_wav))

## MUSDB18 vocal tracks:

### Detect seconds with silence:

In [3]:
dataset = MUSDB18('/home/christosf/AttProtos/datasets/MUSDB18-w44100')

In [4]:
dataset.generate_file_lists()

In [5]:
path = '/home/christosf/AttProtos/datasets/MUSDB18-w44100/'
os.chdir(path)

In [7]:
vocal_paths = []
for dirpath, dirnames, filenames in os.walk(path):
    for filename in filenames:
        if 'vocals.wav' == filename:
            vocal_paths.append(dirpath + '/' + filename)

In [13]:
for path in vocal_paths:
    print(path) 
    sil = []
    sil_path = path.split('/vocals')[0] + "/silences.json"
    print(sil_path)
    myaudio = AudioSegment.from_wav(path)
    dBFS=myaudio.dBFS
    sil = silence.detect_silence(myaudio, min_silence_len=1000, silence_thresh=dBFS-16)
    sil = [((start),(stop)) for start,stop in sil]
    print(sil)
    with open(sil_path, 'w') as f:
        json.dump(sil, f)

/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/test/Secretariat - Over The Top/vocals.wav
/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/test/Secretariat - Over The Top/silences.json
[(0, 28319), (33479, 35134), (39796, 41712), (47125, 48599), (53295, 55289), (56361, 58630), (67147, 84026), (89400, 90729), (95879, 97638), (103091, 104341), (109375, 111070), (112058, 114508), (122788, 130289), (134883, 137112), (140217, 143723), (147954, 150080), (157237, 187118), (188259, 190519), (198928, 202334), (204060, 205803), (207898, 209020), (214096, 250915)]
/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/test/Lyndsey Ollard - Catching Up/vocals.wav
/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/test/Lyndsey Ollard - Catching Up/silences.json
[(0, 14371), (18183, 20647), (23416, 27020), (30649, 33270), (36082, 39677), (42697, 45773), (49990, 52295), (55707, 58407), (61603, 64887), (68651, 71230), (74170, 77158), (81917, 83707), (86910, 90207), (9

### Detect non silent:

In [14]:
for path in vocal_paths:
    print(path) 
    nsil = []
    nsil_path = path.split('/vocals')[0] + "/non_silent.json"
    print(nsil_path)
    myaudio = AudioSegment.from_wav(path)
    dBFS=myaudio.dBFS
    nsil = silence.detect_nonsilent(myaudio, min_silence_len=1000, silence_thresh=dBFS-16)
    nsil = [((start),(stop)) for start,stop in nsil]
    print(nsil)
    with open(nsil_path, 'w') as f:
        json.dump(nsil, f)

/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/test/Secretariat - Over The Top/vocals.wav
/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/test/Secretariat - Over The Top/non_silent.json
[(28319, 33479), (35134, 39796), (41712, 47125), (48599, 53295), (55289, 56361), (58630, 67147), (84026, 89400), (90729, 95879), (97638, 103091), (104341, 109375), (111070, 112058), (114508, 122788), (130289, 134883), (137112, 140217), (143723, 147954), (150080, 157237), (187118, 188259), (190519, 198928), (202334, 204060), (205803, 207898), (209020, 214096)]
/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/test/Lyndsey Ollard - Catching Up/vocals.wav
/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/test/Lyndsey Ollard - Catching Up/non_silent.json
[(14371, 18183), (20647, 23416), (27020, 30649), (33270, 36082), (39677, 42697), (45773, 49990), (52295, 55707), (58407, 61603), (64887, 68651), (71230, 74170), (77158, 81917), (83707, 86910), (90207, 93305), (96460,

# Save timestamps with edges clipped

In [13]:
for dirpath, dirnames, filenames in os.walk(path):
    for filename in filenames:
        if 'vocals.wav' == filename:
            nsil_path = (dirpath + '/non_silent.json')
            print(nsil_path)
            with open(nsil_path, 'r') as f:
                nonsilences = json.load(f)
                sil = [((start-200),(stop+200)) for start,stop in nonsilences]
                print(sil)
                newpath = '/home/christosf/AttProtos/datasets/timestamps/' + dirpath.split('/')[-1] + '_nonsilent.json'
                with open(newpath, 'w') as f2:
                    json.dump(sil, f2)

/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/test/Secretariat - Over The Top/non_silent.json
[(28119, 33679), (34934, 39996), (41512, 47325), (48399, 53495), (55089, 56561), (58430, 67347), (83826, 89600), (90529, 96079), (97438, 103291), (104141, 109575), (110870, 112258), (114308, 122988), (130089, 135083), (136912, 140417), (143523, 148154), (149880, 157437), (186918, 188459), (190319, 199128), (202134, 204260), (205603, 208098), (208820, 214296)]
/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/test/Lyndsey Ollard - Catching Up/non_silent.json
[(14171, 18383), (20447, 23616), (26820, 30849), (33070, 36282), (39477, 42897), (45573, 50190), (52095, 55907), (58207, 61803), (64687, 68851), (71030, 74370), (76958, 82117), (83507, 87110), (90007, 93505), (96260, 99638), (102555, 112076), (115224, 119040), (121368, 126128), (127850, 132847), (133995, 145265), (146029, 150388), (152354, 162996), (165671, 170903), (172044, 176694), (178077, 183492), (184497, 189995),

## SPLIT MIXTURE TRACKS


In [4]:
path = '/home/christosf/AttProtos/datasets/MUSDB18-w44100/original/test'
os.chdir(path)

In [15]:
import math

class SplitWavAudioMubin():
    def __init__(self, folder, new_folder, filename):
        self.folder = folder
        self.new_folder = new_folder
        self.filename = filename
        self.filepath = folder + '/' + filename
        
        self.audio = AudioSegment.from_wav(self.filepath)
    
    def get_duration(self):
        return self.audio.duration_seconds
    
    def split(self, t1, t2, split_filename):
        split_audio = self.audio[t1:t2]
        split_audio.export(self.new_folder + '/' + split_filename, format="wav")
        
    def multiple_split(self, sec_per_split):
        total_secs = math.ceil(self.get_duration())
        for i in range(0, total_secs, sec_per_split):
            split_fn = str(i) + '_' + self.filename
            self.single_split(i, i+sec_per_split, split_fn)
            print(str(i) + ' Done')
            if i == total_secs - sec_per_split:
                print('All splited successfully')

# Split mixture stems: (follow the same method for groundtruth vocals and accompaniment)

In [ ]:
for dirpath, dirnames, filenames in os.walk(path):
    for filename in filenames:
        if 'mixture.wav' == filename:
            wav_path = (dirpath + '/' + filename)
            print(wav_path)
            
            fold = dirpath.split('/')[-2]
            print(fold)
            
            nsil_path = ('/home/christosf/AttProtos/datasets/timestamps/' + dirpath.split('/')[-1] + '_nonsilent.json')
            print(nsil_path)
            
            new_path = '/home/christosf/AttProtos/datasets/MUSDB18Mixtures/' + fold
            with open(nsil_path, 'r') as f:
                nonsilences = json.load(f)
                for index, item in enumerate(nonsilences):
                    t1,t2 = item
                    fn = dirpath.split('/')[-1] + '_vocals_' + '{}'.format(index+1) + '.wav'
                    split_wav = SplitWavAudioMubin(dirpath, new_path, filename)
                    split_wav.split(t1, t2, fn)
            

In [ ]:
for dirpath, dirnames, filenames in os.walk(path):
    for filename in filenames:
        if 'mixture.wav' == filename:
            wav_path = (dirpath + '/' + filename)
            print(wav_path)
            sil_path = ('/home/christosf/AttProtos/datasets/timestamps/' + dirpath.split('/')[-1] +'/silences.json')
            print(sil_path)
            fold = dirpath.split('/')[-2]
            new_path = '/home/christosf/AttProtos/datasets/MUSDB18Mixtures/' + fold
            with open(sil_path, 'r') as f:
                silences = json.load(f)
                for index, item in enumerate(silences):
                    t1,t2 = item
                    fn = dirpath.split('/')[-1] + '_novocals_' + '{}'.format(index+1) + '.wav'
                    split_wav = SplitWavAudioMubin(dirpath, new_path, filename)
                    split_wav.split(t1, t2, fn)

# Create csv tables with annotations

In [5]:
import csv

In [ ]:

path = '/home/christosf/AttProtos/datasets/timestamps/'
recsongs = []
#d = []
for song in songs:
    nsil_path = ('/home/christosf/AttProtos/datasets/timestamps/' + song + '_nonsilent.json')
    sil_path = ('/home/christosf/AttProtos/datasets/timestamps/' + song + '_silences.json')
    chunks = []
    data = []
    with open(nsil_path, 'r') as f:
        nonsilences = json.load(f)
    with open(sil_path, 'r') as f2:
        silences = json.load(f2)
    ind1 = len(silences)
    ind2 = len(nonsilences) 
    
    timestamps1 = silences

    timestamps2 = nonsilences

    for index in range(ind1):
        #print(index)
        ind = index + 1
        if ind == ind1:
            fn1 = '/home/christosf/AttProtos/datasets/MUSDB18Mixtures/groundtruth/' + song + '_novocaltrack_' + '{}'.format(ind) + '/' + song + '_novocaltrack_' + '{}'.format(ind) + '.wav'
            row1 = [song + '_novocals_' + '{}'.format(index) + '.wav', timestamps1[index]]
            data.append(row1)

        else:
            if index < ind1:
                fn2 = '/home/christosf/AttProtos/datasets/MUSDB18Mixtures/groundtruth/' + song + '_vocaltrack_' + '{}'.format(ind) + '/' + song + '_vocaltrack_' + '{}'.format(ind) + '.wav'
                row1 = [song + '_novocals_' + '{}'.format(ind) + '.wav', timestamps1[index]]
                data.append(row1)
                row2 = [song + '_vocals_' + '{}'.format(ind) + '.wav', timestamps2[index]]
                data.append(row2)
    header = ['chunk_name', 'timestamp']
    with open('/home/christosf/AttProtos/datasets/MUSDB18Mixtures/csvs/' + song + '_chunks+timestamps.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(data)



                            